In [1]:
%load_ext autoreload
%autoreload 2

import gymnasium as gym


from src.actor_critic import Mlp, ActorCritic
from src.trainer import PPOModule, PPOModuleConfig
import lightning as pl

In [ ]:
# Initialise the environment
env = gym.make("LunarLander-v3")

# make the actor critic model
d = env.observation_space.shape[0]
n_a = env.action_space.n

actor = Mlp(input_dims=d, output_dims=n_a, hidden_dims=32, n_layers=3)
critic = Mlp(input_dims=d, output_dims=1, hidden_dims=32, n_layers=3)

model = ActorCritic(actor_net=actor, critic=critic)


config = PPOModuleConfig()
trainer = pl.Trainer(max_epochs=1000)
module = PPOModule(actor_critic=model, env=env, config=config)

trainer.fit(module)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type        | Params | Mode 
-----------------------------------------------------
0 | actor_critic | ActorCritic | 189    | train
-----------------------------------------------------
189       Trainable params
0         Non-trainable params
189       Total params
0.001     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode
/Users/nicholasfranklin/miniconda3/envs/state_inference/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/nicholasfranklin/miniconda3/envs/state_inference/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of t

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1000` reached.


In [ ]:
# Initialise the environment
env = gym.make("LunarLander-v3", render_mode="human")

# Reset the environment to generate the first observation
observation, info = env.reset(seed=42)
for _ in range(5000):
    # this is where you would insert your policy
    action = module.predict(observation)

    # step (transition) through the environment with the action
    # receiving the next observation, reward and if the episode has terminated or truncated
    observation, reward, terminated, truncated, info = env.step(action)

    # If the episode has ended then we can reset to start a new episode
    if terminated or truncated:
        observation, info = env.reset()

env.close()

In [4]:
actor

Mlp(
  (input_projection): Identity()
  (output_projection): Linear(in_features=8, out_features=4, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=8, out_features=8, bias=True)
  )
  (act_fn): SiLU()
)